# From Model to BentoML

**Dataset**

In [1]:
!wget https://github.com/gastonstat/CreditScoring/raw/master/CreditScoring.csv --quiet

**Libraries**

In [13]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn import metrics
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_selector as selector
import xgboost as xgb 

In [3]:
df = pd.read_csv('CreditScoring.csv')

In [4]:
# make all columns in lower
df.columns = df.columns.str.lower()

In [5]:
# mapping the target
df['status'] = df['status'].map({
    1:'ok',
    2:'default',
    0:'unk'
})

In [6]:
# mapping categorical features
def mapping_categorical(df, cat, cat_lst):
  to_lst = df[cat].value_counts().sort_index().index.to_list()
  cat_lst = cat_lst

  df[cat] = (
      df[cat].map({
          k:v for (k,v) in zip(to_lst, cat_lst)
      })
   )

cols = ['home', 'marital', 'records', 'job']

home_lst = ['unk', 'rent', 'owner', 'private', 'ignore', 'parents', 'other']
marital_lst = ['unk', 'single', 'married', 'widow', 'separated', 'divorced']
records_lst = ['no', 'yes', 'unk']
job_lst = ['unk', 'fixed', 'partime', 'freelance', 'others']
cat_lst = [home_lst, marital_lst, records_lst, job_lst]

for col, cat in zip(cols, cat_lst):
  mapping_categorical(df, col, cat)

In [7]:
# fix missing values
def fix_missing_values(df, val_to_rep, rep, *f_lst):
  for f in f_lst:
    df[f] = df[f].replace(val_to_rep, rep)

fix_missing_values(df, 99999999.0, np.nan, ['income', 'assets', 'debt'])

In [8]:
# don't nees unk in status
df = df[df.status != 'unk']

In [10]:
# data preparation
data, target = df.drop(columns=['status']), df['status'].map({'ok':0, 'default':1})

def tweaking(data, target):
  numerical = selector(dtype_include=np.number)(data)
  categorical = selector(dtype_include=object)(data)

  num_imputer = SimpleImputer(missing_values=np.NaN, strategy='constant', fill_value=0)
  cat_imputer = SimpleImputer(strategy='most_frequent', fill_value='unk')
  cat_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
  
  X_full_train, X_test, y_full_train, y_test = model_selection.train_test_split(
      data,
      target,
      test_size=.2,
      random_state=11,
    )
  X_train, X_dev, y_train, y_dev = model_selection.train_test_split(
          X_full_train,
          y_full_train,
          test_size=.25,
          random_state=11,
        )


  X_train.loc[:, numerical] = num_imputer.fit_transform(X_train[numerical])
  X_train.loc[:, categorical] = cat_imputer.fit_transform(X_train[categorical])
  X_train.loc[:, categorical] = cat_encoder.fit_transform(X_train[categorical])

  X_dev.loc[:, numerical] = num_imputer.transform(X_dev[numerical])
  X_dev.loc[:, categorical] = cat_imputer.transform(X_dev[categorical])
  X_dev.loc[:, categorical] = cat_encoder.transform(X_dev[categorical])

  X_test.loc[:, numerical] = num_imputer.transform(X_test[numerical])
  X_test.loc[:, categorical] = cat_imputer.transform(X_test[categorical])
  X_test.loc[:, categorical] = cat_encoder.transform(X_test[categorical])

  return X_train, y_train, X_dev, y_dev, X_test, y_test

In [12]:
X_train, y_train, X_dev, y_dev, X_test, y_test = tweaking(data, target)

In [14]:
# wrap data into DMatrix — a special
# data structure for finding splits efficiently.
dtrain = xgb.DMatrix(
    X_train.values, 
    label=y_train.values, 
    feature_names=X_train.columns
)

# for validation
dval = xgb.DMatrix(
    X_dev.values,
    label=y_dev.values,
    feature_names=X_dev.columns
)

# specifying the parameters for training
xgb_params = {
    'eta':.3,
    'max_depth':6,
    'min_child_weight': 1,
    'objective': 'binary:logistic',
    'nthread': -1,
    'seed': 1,
    'silent':1
}

# For training an XGBoost model, we use the train function
model = xgb.train(
    xgb_params,
    dtrain,
    num_boost_round=10
)

y_pred = model.predict(dval)
metrics.roc_auc_score(y_dev, y_pred)

0.8121897023564457

In [15]:
watchlist = [(dtrain, 'train'), (dval, 'dev')]

xgb_params = {
    'eta':.05,
    'max_depth':3,
    'min_child_weight': 30,
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'nthread': -1,
    'seed': 1,
    'silent':1
}

model = xgb.train(
    xgb_params,
    dtrain,
    num_boost_round=500,
    evals=watchlist,
    verbose_eval=10
)

[0]	train-auc:0.766001	dev-auc:0.734594
[10]	train-auc:0.82062	dev-auc:0.769786
[20]	train-auc:0.836773	dev-auc:0.785713
[30]	train-auc:0.846398	dev-auc:0.79379
[40]	train-auc:0.85774	dev-auc:0.806181
[50]	train-auc:0.86499	dev-auc:0.812032
[60]	train-auc:0.869141	dev-auc:0.815054
[70]	train-auc:0.873505	dev-auc:0.819407
[80]	train-auc:0.876601	dev-auc:0.822128
[90]	train-auc:0.878781	dev-auc:0.822334
[100]	train-auc:0.880879	dev-auc:0.824502
[110]	train-auc:0.882434	dev-auc:0.826506
[120]	train-auc:0.883788	dev-auc:0.827841
[130]	train-auc:0.88528	dev-auc:0.82841
[140]	train-auc:0.886643	dev-auc:0.8293
[150]	train-auc:0.887832	dev-auc:0.829863
[160]	train-auc:0.889011	dev-auc:0.830801
[170]	train-auc:0.89029	dev-auc:0.831643
[180]	train-auc:0.891469	dev-auc:0.832218
[190]	train-auc:0.892371	dev-auc:0.832418
[200]	train-auc:0.893084	dev-auc:0.832757
[210]	train-auc:0.893869	dev-auc:0.833054
[220]	train-auc:0.894574	dev-auc:0.833175
[230]	train-auc:0.895326	dev-auc:0.833357
[240]	train-

In [16]:
dtest = xgb.DMatrix(
    X_test.values,
    label=y_test.values, 
    feature_names=X_test.columns
)

y_pred_dev = model.predict(dval)
y_pred_test = model.predict(dtest)

print(f"AUC-dev: {metrics.roc_auc_score(y_dev, y_pred_dev): .3f}")
print(f"AUC-test: {metrics.roc_auc_score(y_test, y_pred_test): .3f}")

AUC-dev:  0.833
AUC-test:  0.820
